# Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models.data_split import MidiDataset, VOCABULARY
from models.model import NoteComposeNet
from torch.utils.data import DataLoader
from train import TrainPipeline

import torch
import pandas as pd

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from datetime import datetime

import math

In [3]:
BATCH_SIZE = 32
TRAIN_SAMPLES_PER_TRACK = int(1e6)
VALIDATE_SAMPLES_PER_TRACK = int(1e6 // 8)
EPOCHS = 20
EPOCHS_SO_FAR = 0
GRADIENT_ACC = 16

In [4]:
CHECKPOINT = torch.load(r'checkpoints/model_20230905_160343_9')
model = NoteComposeNet()
model.load_state_dict(CHECKPOINT)

param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 0.386MB


In [5]:
CSV_PATH = r'datasets/midi-dataset-flat.csv'
df = pd.read_csv(CSV_PATH)
train_midi = MidiDataset(df, context_len = model._context_len, train_samples=TRAIN_SAMPLES_PER_TRACK, validate_samples=VALIDATE_SAMPLES_PER_TRACK, start_length=-1)
del df

In [6]:
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0)

optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=0.0001,
    weight_decay=0.1, 
    betas=(0.9, 0.95),
    eps=1e-05,
    amsgrad=False
    )

scheduler = CosineAnnealingWarmRestarts(
    optimizer=optimizer, 
    T_0= int(math.ceil(len(train_midi) / (GRADIENT_ACC * BATCH_SIZE))),
    eta_min=1e-6,
)

In [7]:
print("Total samples: ", EPOCHS * len(train_midi))
print("Total tokens:", EPOCHS * len(train_midi) * 2048)
print("Total updates per epoch", int(math.ceil(len(train_midi) / (GRADIENT_ACC * BATCH_SIZE))))

Total samples:  20000000
Total tokens: 40960000000
Total updates per epoch 1954


In [8]:
pipeline = TrainPipeline(train_midi, model, loss_fn, optimizer, validate=True, 
                         batch_size=BATCH_SIZE, scheduler = scheduler, grad_acc = GRADIENT_ACC)
pipeline.train(EPOCHS)

Epoch 11:  43%|████▎     | 13407/31250 [15:22<20:46, 14.32batch/s, train loss=3.14869, lr=[6.23223478479707e-05]] 